In [142]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import numpy.typing as npt
import cProfile

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


# create a seed
seed = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=np.int8)

# create a biome
biome = np.zeros((100, 100), dtype=np.int8)
biome[0:10, 0:10] = seed




In [117]:
class Biome():
    def __init__(self, grid: npt.NDArray[np.int8]):
        self.grid = grid

    def run(self, generations: int):
        for i in range(generations):
            self.grid = self._update(self.grid)
        
    def _update(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        
        if np.sum(grid) == 0:
            return grid

        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([indices + np.array([i, j]) for i in range(-1, 2) for j in range(-1, 2) if indices.shape[0] != 0])

        # colapse the array
        surrounding_indices = surrounding_indices.reshape(-1, 2)
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]

        # remove the indices that are the same as the original indices
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] != indices[:, 0]) | (surrounding_indices[:, 1] != indices[:, 1])]

        # for each surrounding index that occures three times, set the biome to 1 
        for i in np.unique(surrounding_indices, axis=0):
            if np.sum(np.all(surrounding_indices == i, axis=1)) == 3:
                grid[i[0], i[1]] = 1
        
        # set indices that are not surrounded by 2 or 3 cells to 0
        for i in np.unique(indices, axis=0):
            if np.sum(np.all(surrounding_indices == i, axis=1)) not in [2, 3]:
                grid[i[0], i[1]] = 0
        
        print(grid)

biome = Biome(biome)
biome.run(100)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 

AttributeError: 'NoneType' object has no attribute 'shape'

In [194]:
# the rules of connways game of life
# any live cell with fewer than two live neighbours dies, as if by underpopulation.
# any live cell with two or three live neighbours lives on to the next generation.
# any live cell with more than three live neighbours dies, as if by overpopulation.
# any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

grid = np.zeros((100, 100), dtype=np.int8)
grid[0:10, 0:10] = seed
# get indices of all non-zero elements
indices = np.argwhere(grid == 1)
print(grid[:10, :10])
# get all indices at [-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1] from the indices
surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])

        # remove the indices that are out of bounds
surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]

# for each surrounding index that occures three times, set the biome to 1
for i in np.unique(surrounding_indices, axis=0):
        if np.sum(np.all(surrounding_indices == i, axis=1)) == 3:
                grid[i[0], i[1]] = 1
        elif grid[i[0], i[1]] == 1 & np.sum(np.all(surrounding_indices == i, axis=1)) == 2:
                grid[i[0], i[1]] = 1
        else:
                grid[i[0], i[1]] = 0

print(grid[:10, :10])

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [177]:

[1,1] + [1,1]

[1, 1, 1, 1]